In [1]:
!pip install -q torch torchvision kagglehub
!pip install -q pytorch-lightning torchmetrics "numpy<2.0"
!pip install -q pandas matplotlib seaborn
!pip install -q grad-cam

In [2]:
import os
import json
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import torchmetrics
import kagglehub
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-3
NUM_WORKERS = 4
SEED = 1234

CHECKPOINT_BEST = "best_model.ckpt"
CLASSES_FILE = "classes.json"
REPORT_FILE = "raport_aut.csv"
PREDICTIONS_FILE = "wyniki_efficientnet.csv"

pl.seed_everything(SEED, workers=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PIN_MEMORY = DEVICE.type == "cuda"

def find_train_test_dirs(base_dir: str):
    for root, dirs, _files in os.walk(base_dir):
        if "train" in dirs and "test" in dirs:
            candidate_train = os.path.join(root, "train")
            candidate_test = os.path.join(root, "test")
            has_class_dirs = any(
                os.path.isdir(os.path.join(candidate_train, d))
                for d in os.listdir(candidate_train)
            )
            if has_class_dirs:
                return candidate_train, candidate_test
    return None, None

def dataloader_kwargs():
    kwargs = {"num_workers": NUM_WORKERS, "pin_memory": PIN_MEMORY}
    if NUM_WORKERS and NUM_WORKERS > 0:
        kwargs["persistent_workers"] = True
    return kwargs

def make_loader(dataset, *, shuffle: bool):
    return DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=shuffle, **dataloader_kwargs())

def make_transforms(mode: str):
    mode = mode.lower()
    if mode == "train":
        return transforms.Compose(
            [
                transforms.Resize((256, 256)),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(0.2, 0.2),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
    if mode in {"val", "test", "eval"}:
        return transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
    raise ValueError(f"Nieznany tryb transformacji: {mode}")

def load_eval_model(checkpoint_path: str, *, num_classes: int):
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Brak checkpointu: {checkpoint_path}")
    model = CarClassifier.load_from_checkpoint(
        checkpoint_path,
        num_classes=num_classes,
        learning_rate=LEARNING_RATE,
        weights=None,
        trainable_blocks=0,
    )
    model.to(DEVICE)
    model.eval()
    model.freeze()
    return model

BASE_DIR = kagglehub.dataset_download("jutrera/stanford-car-dataset-by-classes-folder")
train_dir, test_dir = find_train_test_dirs(BASE_DIR)
if not train_dir:
    raise FileNotFoundError("Nie znaleziono folderów train/test w dataset.")

print(f"Trening: {train_dir}")
print(f"Test:    {test_dir}")

Seed set to 1234


Trening: /teamspace/studios/this_studio/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/car_data/train
Test:    /teamspace/studios/this_studio/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/car_data/test


In [11]:
class CarClassifier(pl.LightningModule):
    def __init__(
        self,
        num_classes: int,
        learning_rate: float = 1e-3,
        weights: str | None = "DEFAULT",
        trainable_blocks: int = 3,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.lr = learning_rate

        self.model = models.efficientnet_v2_s(weights=weights)

        if trainable_blocks is not None:
            for param in self.model.features.parameters():
                param.requires_grad = False
            if trainable_blocks > 0:
                for param in self.model.features[-trainable_blocks:].parameters():
                    param.requires_grad = True

        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, _batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.train_acc(torch.argmax(logits, dim=1), y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_acc", self.train_acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, _batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.val_acc(torch.argmax(logits, dim=1), y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        params = [{"params": self.model.classifier.parameters(), "lr": self.lr}]
        if getattr(self.hparams, "trainable_blocks", 0) and self.hparams.trainable_blocks > 0:
            params.insert(
                0,
                {
                    "params": self.model.features[-self.hparams.trainable_blocks :].parameters(),
                    "lr": self.lr * 0.1,
                },
            )
        optimizer = optim.Adam(params)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.1, patience=2
        )
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

In [ ]:
train_tf = make_transforms("train")
eval_tf = make_transforms("eval")

train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds = datasets.ImageFolder(test_dir, transform=eval_tf)

train_loader = make_loader(train_ds, shuffle=True)
val_loader = make_loader(val_ds, shuffle=False)

class_names = train_ds.classes
with open("classes.json", "w", encoding="utf-8") as f:
    json.dump(class_names, f, ensure_ascii=False)

print(f"Liczba klas: {len(class_names)}")
print(f"Train: {len(train_ds)} zdjęć")
print(f"Val:   {len(val_ds)} zdjęć")

In [ ]:
print("Start treningu...")

csv_logger = pl.loggers.CSVLogger(save_dir="logs", name="efficientnet_experiment")
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=".",
    filename="best_model",
    save_top_k=1,
    monitor="val_acc",
    mode="max",
)

trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator="auto",
    devices=1,
    callbacks=[checkpoint_callback],
    logger=csv_logger,
    enable_progress_bar=True,
)

model = CarClassifier(num_classes=len(class_names), learning_rate=LEARNING_RATE)
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

print(f"Trening zakończony. Najlepszy checkpoint: {checkpoint_callback.best_model_path}")

In [ ]:
metrics_path = f"{csv_logger.log_dir}/metrics.csv"
if not os.path.exists(metrics_path):
    print("Nie znaleziono metrics.csv (brak logów).")
else:
    metrics = pd.read_csv(metrics_path)
    epoch_metrics = metrics.groupby("epoch").mean(numeric_only=True)

    sns.set(style="whitegrid")
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    axes[0].plot(epoch_metrics.index, epoch_metrics["train_loss"], marker="o", label="Train")
    axes[0].plot(epoch_metrics.index, epoch_metrics["val_loss"], marker="o", label="Val")
    axes[0].set_title("Loss")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")
    axes[0].legend()

    axes[1].plot(epoch_metrics.index, epoch_metrics["train_acc"], marker="o", label="Train")
    axes[1].plot(epoch_metrics.index, epoch_metrics["val_acc"], marker="o", label="Val")
    axes[1].set_title("Accuracy")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Accuracy")
    axes[1].legend()

    plt.tight_layout()
    plt.savefig("training_curves.png", dpi=150)
    plt.show()
    print("Zapisano: training_curves.png")

In [ ]:
checkpoint_path = checkpoint_callback.best_model_path or CHECKPOINT_BEST
model = load_eval_model(checkpoint_path, num_classes=len(class_names))

test_ds = datasets.ImageFolder(test_dir, transform=eval_tf)
loader = make_loader(test_ds, shuffle=False)

file_paths = [s[0] for s in test_ds.samples]
rows = []

print(f"Analiza {len(test_ds)} zdjęć (EfficientNet)...")
for batch_idx, (inputs, labels) in enumerate(tqdm(loader, desc="Test")):
    inputs = inputs.to(DEVICE)
    preds = model(inputs)
    probs = torch.softmax(preds, dim=1)
    scores, pred_ids = torch.max(probs, dim=1)

    start_i = batch_idx * loader.batch_size
    for i in range(len(inputs)):
        idx_global = start_i + i
        if idx_global >= len(file_paths):
            break

        path = file_paths[idx_global]
        filename = os.path.basename(path)
        true_name = class_names[labels[i].item()]
        pred_name = class_names[pred_ids[i].item()]
        score_pct = float(scores[i].item()) * 100

        rows.append(
            {
                "Plik": filename,
                "Prawdziwe Auto": true_name,
                "Wykryte Auto": pred_name,
                "Pewnosc Modelu": f"{score_pct:.2f}%",
                "Czy Trafil": "TAK" if true_name == pred_name else "NIE",
            }
)

df = pd.DataFrame(rows)
acc = (df["Czy Trafil"] == "TAK").mean() * 100

print(f"Skuteczność: {acc:.2f}%")
df.to_csv(PREDICTIONS_FILE, index=False, encoding="utf-8")
df.drop(columns=["Plik"]).to_csv(REPORT_FILE, index=False, encoding="utf-8")
print(f"Zapisano: {PREDICTIONS_FILE}")
print(f"Zapisano: {REPORT_FILE}")

df.head(20)

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 1000)
pd.set_option("display.precision", 2)

COMPARISON_OUTPUT_FILE = "porownanie_modeli_klasy.csv"

def generate_report(csv_path: str, model_name: str):
    if not os.path.exists(csv_path):
        print(f"Brak pliku: {csv_path}")
        return None

    df_local = pd.read_csv(csv_path)
    report = df_local.groupby("Prawdziwe Auto").agg(
        photos_count=("Plik", "count"),
        guesses=("Czy Trafil", lambda x: (x == "TAK").sum()),
)
    report["accuracy"] = report["guesses"] / report["photos_count"] * 100
    report = report.sort_values(by="accuracy", ascending=True)
    print(f"\n{model_name}: (najgorsze klasy na górze)")
    print(report[["photos_count", "accuracy"]].head(20))
    return report[["accuracy"]].rename(columns={"accuracy": model_name})

eff_stats = generate_report("wyniki_efficientnet.csv", "EfficientNet")
conv_stats = generate_report("wyniki_convnext.csv", "ConvNeXt")

if eff_stats is not None and conv_stats is not None:
    comparison = pd.concat([eff_stats, conv_stats], axis=1)
    comparison["Różnica"] = comparison["ConvNeXt"] - comparison["EfficientNet"]
    comparison = comparison.sort_values(by="Różnica", ascending=False)
    print("\nTop różnic (ConvNeXt - EfficientNet):")
    print(comparison.head(30))
    comparison.to_csv(COMPARISON_OUTPUT_FILE, encoding="utf-8")
    print(f"\nZapisano: {COMPARISON_OUTPUT_FILE}")

In [ ]:
EFF_CHECKPOINT = "best_model.ckpt"
CONV_CHECKPOINT = "best_convnext.ckpt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CarClassifierEffNetEval(pl.LightningModule):
    def __init__(self, num_classes: int):
        super().__init__()
        self.model = models.efficientnet_v2_s(weights=None)
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

class CarClassifierConvNextEval(pl.LightningModule):
    def __init__(self, num_classes: int):
        super().__init__()
        self.model = models.convnext_tiny(weights=None)
        in_features = self.model.classifier[2].in_features
        self.model.classifier[2] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

model_eff = None
if os.path.exists(EFF_CHECKPOINT):
    model_eff = CarClassifierEffNetEval.load_from_checkpoint(EFF_CHECKPOINT, num_classes=len(class_names))
    model_eff.eval().to(device)
else:
    print(f"Brak checkpointu EfficientNet: {EFF_CHECKPOINT}")

model_conv = None
if os.path.exists(CONV_CHECKPOINT):
    model_conv = CarClassifierConvNextEval.load_from_checkpoint(CONV_CHECKPOINT, num_classes=len(class_names))
    model_conv.eval().to(device)
else:
    print(f"Brak checkpointu ConvNeXt: {CONV_CHECKPOINT}")

def visualize_comparison(image_path: str):
    img_pil = Image.open(image_path).convert("RGB").resize((224, 224))
    rgb_img = np.float32(img_pil) / 255.0
    input_tensor = preprocess_image(
        rgb_img,
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ).to(device)

    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    axs[0].imshow(img_pil)
    axs[0].set_title(os.path.basename(image_path))
    axs[0].axis("off")

    if model_eff is not None:
        cam_eff = GradCAM(model=model_eff, target_layers=[model_eff.model.features[-1]])
        grayscale_cam_eff = cam_eff(input_tensor=input_tensor, targets=None)
        vis_eff = show_cam_on_image(rgb_img, grayscale_cam_eff[0, :], use_rgb=True)

        preds_eff = model_eff(input_tensor)
        idx_eff = int(torch.argmax(preds_eff, dim=1).item())
        conf_eff = float(torch.softmax(preds_eff, dim=1)[0, idx_eff].item())

        axs[1].imshow(vis_eff)
        axs[1].set_title(f"EfficientNet: {class_names[idx_eff]} ({conf_eff:.1%})")
        axs[1].axis("off")
    else:
        axs[1].text(0.5, 0.5, "Brak EfficientNet", ha="center", va="center")
        axs[1].axis("off")

    if model_conv is not None:
        cam_conv = GradCAM(model=model_conv, target_layers=[model_conv.model.features[-1]])
        grayscale_cam_conv = cam_conv(input_tensor=input_tensor, targets=None)
        vis_conv = show_cam_on_image(rgb_img, grayscale_cam_conv[0, :], use_rgb=True)

        preds_conv = model_conv(input_tensor)
        idx_conv = int(torch.argmax(preds_conv, dim=1).item())
        conf_conv = float(torch.softmax(preds_conv, dim=1)[0, idx_conv].item())

        axs[2].imshow(vis_conv)
        axs[2].set_title(f"ConvNeXt: {class_names[idx_conv]} ({conf_conv:.1%})")
        axs[2].axis("off")
    else:
        axs[2].text(0.5, 0.5, "Brak ConvNeXt", ha="center", va="center")
        axs[2].axis("off")

    plt.tight_layout()
    plt.show()

test_images = [p for p, _ in datasets.ImageFolder(test_dir, transform=make_transforms("eval")).samples]
sample = random.sample(test_images, k=min(5, len(test_images)))
print(f"Wybrano {len(sample)} obrazów do porównania.")
for img_path in sample:
    visualize_comparison(img_path)